In [1]:
import numpy as np
import pandas as pd
import pickle 

In [2]:
df = pickle.load(open('dataset_level2.pkl','rb'))
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,2,Australia,Pakistan,0.1,0,0,Brisbane,"Brisbane Cricket Ground, Woolloongabba"
1,2,Australia,Pakistan,0.2,0,0,Brisbane,"Brisbane Cricket Ground, Woolloongabba"
2,2,Australia,Pakistan,0.3,0,0,Brisbane,"Brisbane Cricket Ground, Woolloongabba"
3,2,Australia,Pakistan,0.4,0,0,Brisbane,"Brisbane Cricket Ground, Woolloongabba"
4,2,Australia,Pakistan,0.5,1,0,Brisbane,"Brisbane Cricket Ground, Woolloongabba"


In [3]:
df.isnull().sum()

match_id                0
batting_team            0
bowling_team            0
ball                    0
runs                    0
player_dismissed        0
city                59517
venue                   0
dtype: int64

In [4]:
df[df['city'].isnull()]['venue'].value_counts()

venue
Sydney Cricket Ground                      10301
Rangiri Dambulla International Stadium     10051
Melbourne Cricket Ground                    8391
Adelaide Oval                               7725
Pallekele International Cricket Stadium     6170
Dubai International Cricket Stadium         6008
Sharjah Cricket Stadium                     4363
Harare Sports Club                          1839
Multan Cricket Stadium                      1199
Queenstown Events Centre                     909
Chittagong Divisional Stadium                900
Rawalpindi Cricket Stadium                   617
Perth Stadium                                526
Dubai Sports City Cricket Stadium            518
Name: count, dtype: int64

In [5]:
cities = np.where(df['city'].isnull(),df['venue'].str.split().apply(lambda x:x[0]),df['city'])
df['city'] = cities

In [6]:
df.isnull().sum()

match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [7]:
df.drop(columns=['venue'],inplace=True)
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Australia,Pakistan,0.1,0,0,Brisbane
1,2,Australia,Pakistan,0.2,0,0,Brisbane
2,2,Australia,Pakistan,0.3,0,0,Brisbane
3,2,Australia,Pakistan,0.4,0,0,Brisbane
4,2,Australia,Pakistan,0.5,1,0,Brisbane


In [8]:
eligible_cities = df['city'].value_counts()[df['city'].value_counts() > 600].index.tolist()
df = df[df['city'].isin(eligible_cities)]

In [9]:
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Australia,Pakistan,0.1,0,0,Brisbane
1,2,Australia,Pakistan,0.2,0,0,Brisbane
2,2,Australia,Pakistan,0.3,0,0,Brisbane
3,2,Australia,Pakistan,0.4,0,0,Brisbane
4,2,Australia,Pakistan,0.5,1,0,Brisbane


In [10]:
df['current_score'] = df.groupby('match_id')['runs'].cumsum()
df['over'] = df['ball'].apply(lambda x:str(x).split(".")[0])
df['ball_no'] = df['ball'].apply(lambda x:str(x).split(".")[1])
df['balls_bowled'] = (df['over'].astype('int')*6) + df['ball_no'].astype('int')
df['balls_left'] = 120 - df['balls_bowled']
df['balls_left'] = df['balls_left'].apply(lambda x:0 if x<0 else x)
df['player_dismissed'] = df['player_dismissed'].apply(lambda x:0 if x=='0' else 1)
df['player_dismissed'] = df['player_dismissed'].astype('int')
df['player_dismissed'] = df.groupby('match_id')['player_dismissed'].cumsum()
df['wickets_left'] = 10 - df['player_dismissed']

In [11]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,current_score,over,ball_no,balls_bowled,balls_left,wickets_left
0,2,Australia,Pakistan,0.1,0,0,Brisbane,0,0,1,1,119,10
1,2,Australia,Pakistan,0.2,0,0,Brisbane,0,0,2,2,118,10
2,2,Australia,Pakistan,0.3,0,0,Brisbane,0,0,3,3,117,10
3,2,Australia,Pakistan,0.4,0,0,Brisbane,0,0,4,4,116,10
4,2,Australia,Pakistan,0.5,1,0,Brisbane,1,0,5,5,115,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
693917,2424,Sri Lanka,Australia,39.5,0,9,Pallekele,194,39,5,239,0,1
693918,2424,Sri Lanka,Australia,39.6,0,9,Pallekele,194,39,6,240,0,1
693919,2424,Sri Lanka,Australia,39.7,1,9,Pallekele,195,39,7,241,0,1
693920,2424,Sri Lanka,Australia,40.1,0,9,Pallekele,195,40,1,241,0,1


In [12]:
df['crr'] = (df['current_score']*6)/df['balls_bowled']

In [13]:
groups = df.groupby('match_id')

match_ids = df['match_id'].unique()
last_five = []

for id in match_ids:
    runs_rolling = groups.get_group(id)['runs'].rolling(window=30).sum()
    last_five.extend(runs_rolling.values.tolist())

df['last_five'] = last_five


In [14]:
final_df = df.groupby('match_id').sum()['runs'].reset_index().merge(df,on='match_id')

In [15]:
final_df=final_df[['batting_team','bowling_team','city','current_score','balls_left','wickets_left','crr','last_five','runs_x']]

In [16]:
final_df.dropna(inplace=True)

In [17]:
final_df.isnull().sum()

batting_team     0
bowling_team     0
city             0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_five        0
runs_x           0
dtype: int64

In [18]:
final_df = final_df.sample(final_df.shape[0])
final_df.sample(2)

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
247880,West Indies,Bangladesh,Dominica,141,0,6,4.188119,19.0,274
253436,Australia,Pakistan,Perth,30,82,10,4.736842,29.0,277


In [19]:
final_df=final_df[['batting_team','bowling_team','city','current_score','balls_left','wickets_left','crr','last_five','runs_x']]

In [20]:
final_df.head()

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
344437,New Zealand,Sri Lanka,Pallekele,28,62,9,2.896552,16.0,250
29410,Australia,India,Bengaluru,84,36,10,6.000000,30.0,334
307930,Australia,India,Adelaide,221,0,6,5.282869,35.0,269
250081,Sri Lanka,New Zealand,Colombo,183,0,5,4.021978,35.0,216
157741,Sri Lanka,South Africa,Perth,125,5,9,6.521739,45.0,221


In [21]:
pickle.dump(final_df,open('dataset_final.pkl','wb'))